## 데이터 작업
- torch.utils.data.Dataset 
    - sample, label 저장
    - sample, label 변경하기 위한 transform, target_transform 인자 포함


- torch.utils.data.Dataloader 
    - dataset을 iterable로 감싼다


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

c:\Users\User\anaconda3\envs\kaggle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## 데이터 불러오기
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
# Dataset을 Dataloader의 인자로 전달: datset을 iterable로 감싸고, batch, sampling, shuffling 등등 다중 프로세스로 데이터 불러오기 지원(multiprocess data loading)
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 모델 생성
- nn.Module을 상속받는 클래스를 생성하여 정의함

In [6]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 변수 최적화
- optimizer, loss function 이용

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
# 학습
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad() # 모델 파라미터 gradient를 0으로 초기화
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
# 모델 성능 평가
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.318397  [   64/60000]
loss: 2.295876  [ 6464/60000]
loss: 2.277198  [12864/60000]
loss: 2.260147  [19264/60000]
loss: 2.250931  [25664/60000]
loss: 2.233668  [32064/60000]
loss: 2.222610  [38464/60000]
loss: 2.203509  [44864/60000]
loss: 2.202269  [51264/60000]
loss: 2.164978  [57664/60000]
Test Error: 
 Accuracy: 49.2%, Avg loss: 2.160675 

Epoch 2
-------------------------------
loss: 2.181684  [   64/60000]
loss: 2.162536  [ 6464/60000]
loss: 2.105474  [12864/60000]
loss: 2.111271  [19264/60000]
loss: 2.055572  [25664/60000]
loss: 2.010034  [32064/60000]
loss: 2.026448  [38464/60000]
loss: 1.954742  [44864/60000]
loss: 1.964829  [51264/60000]
loss: 1.886287  [57664/60000]
Test Error: 
 Accuracy: 50.8%, Avg loss: 1.883404 

Epoch 3
-------------------------------
loss: 1.927185  [   64/60000]
loss: 1.890443  [ 6464/60000]
loss: 1.773834  [12864/60000]
loss: 1.808184  [19264/60000]
loss: 1.683389  [25664/60000]
loss: 1.653441  [32064/600

## 모델 저장
모델의 interal state dictionary를 직렬화(serialize) 해서 저장

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
# 불러온 모델로 예측 수행
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # 모델을 평가 모드로 전환
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad(): # gradient 학습 없음
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
